In [274]:
import pandas as pd

In [428]:
### Pulled this data from https://www.climatetrace.org/inventory in 02/2022
#### Disclaimer from the source that should be noted: "All Sectors excluding Forest and Land Use Emissions"
df = pd.read_csv("climatetrace_emissions_by_subsector_timeseries_all.csv")

##Manipulations for climate tace data cleaning
df = df.rename(columns = {'Tonnes Co2e':'Tonnes_C02'})
df = df.fillna(0)
df['Tonnes_C02_Millions'] = df['Tonnes_C02']/1000000
df['year'] = pd.DatetimeIndex(df['start']).year
df = df.rename(columns = {'country':'country_code'})
df.year = df.year.astype(str)

### Pulled this data from https://data.worldbank.org/indicator/SP.POP.TOTL in 02/2022
wpop = pd.read_csv("API_SP.POP.TOTL_DS2_en_csv_v2_3628828.csv",skiprows=3)
wpop = wpop.rename(columns={'Country Code':'country_code'})
wpop = wpop.fillna(0)

##Changes to the dataframe so I can easily merge it with the other dataframe
keepcolumns = ['Country Name','country_code','Indicator Name','Indicator Code']
value_columns = wpop.columns.difference(keepcolumns)
wpop = pd.melt(wpop,id_vars=keepcolumns,value_vars=value_columns,var_name='Year',
        value_name='Population')
wpop = wpop[['Country Name','country_code','Year','Population']]

###Merging the two dataframes and a quick per capita calculation
df = pd.merge(df,wpop,how='left',left_on=['country_code','year'],right_on=['country_code','Year'])
df.reset_index(drop=True,inplace=True)
df['Pounds_C02_PerCapita'] = df['Tonnes_C02']/df['Population']*2000

###Bringing in Groupings from the world bank to category and remove outliers
wg = pd.read_csv('Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2_3628828.csv')
wg = wg[['Country Code','Region','IncomeGroup']]
df = pd.merge(df,wg,how='left',left_on='country_code',right_on='Country Code')
df = df.drop(columns=['Country Code'])

df.to_csv('Yearly C02 emissions - Per Country and Per Capita 2015-2020.csv')

